# Azure OpenAI Quickstart

## Overview  
> "Large Language Model(LLM)은 주어진 텍스트 다음에 나올 텍스트를 토큰 단위로 생성하는 모델입니다. 입력 문자열이 주어지면 LLM은 다음에 나올 텍스트를 예측합니다".  

이 `Azure OpenAI Quickstarts` 노트북은 사용자에게 LLM 개념을 설명하고 LLM을 활용하기 위한 핵심 패키지 요구 사항, 프롬프트 디자인에 대한 간략한 소개 및 다양한 사용 사례에 대한 몇 가지 간단한 예를 소개합니다. 더 많은 빠른 시작 예제를 보려면 [공식 Azure Open AI 빠른 시작 설명서](https://learn.microsoft.com/en-us/azure/ai-services/openai/quickstart?pivots=programming-language-studio)를 참조하세요.  
  
해당 컨텐츠는 빠르게 실습을 진행할 수 있도록 컨테이너 기반으로 환경이 모두 설치가 된 상태로 시작할 수 있습니다. 따라서, [DevContainer](https://code.visualstudio.com/docs/devcontainers/containers) 기반으로 개발에 필요한 환경설정이 사전 정의되어 있습니다. GitHub Codespace를 활용하거나, 로컬에 Docker를 설치한 상태에서 Visual Stduio Code IDE에 해당 Repository를 다운로드 받을 경우, 자동으로 컨테이너에 개발환경(Python Runtime 3.11.4, Azure OpenAI 1.13.3)을 설치합니다. .env 파일에 필수 API 정보를 입력 하고 저장 후 사용하세요.

### Getting started with Azure OpenAI Service

신규 고객은 Azure OpenAi 서비스에 [액세스 신청](https://aka.ms/oai/access)을 해야합니다.
승인이 완료된 후 고객은 Azure Portal에 로그인하고 Azure OpenAI 서비스 리소스를 만들고 스튜디오를 통해 모델 실험을 시작할 수 있습니다.

[Great resource for getting started quickly](https://techcommunity.microsoft.com/t5/educator-developer-blog/azure-openai-is-now-generally-available/ba-p/3719177 )


## Build your first prompt  
이 짧은 연습은 간단한 작업 "요약"을 위해 OpenAI 모델에 프롬프트를 제출하기위한 기본 소개를 제공합니다.

![](images/generative-AI-models-reduced.jpg)  


**Steps**:  
1. 파이썬 환경에 OpenAI 라이브러리를 설치
2. 표준 헬퍼 라이브러리를 로드하고 OpenAI 보안 자격 증명을 설정
3. OpenAI 작업에 적합한 모델을 선택
4. 모델에 대한 간단한 프롬프트를 만듭니다.
5. 모델 API에 요청을 제출하십시오!

## 1. 파이썬 환경에 OpenAI 라이브러리를 설치
DevContainer 가 시작할 때, 자동으로 `requirements.txt` 에 기술한 라이브러리를 설치합니다.
따라서 추가적으로 라이브러리 설치를 진행하지 않아도 즉시 실습할 수 있습니다.
Python 버전은 **Python 3.11.4**, **Azure OpenAI 1.13.3** 를 사용합니다.

## 2. 표준 헬퍼 라이브러리를 로드하고 OpenAI 보안 자격 증명을 설정
루트 디렉토리에 존재하는 `.evn.sample` 파일을 복사하여 `.env` 파일로 이름을 변경하고, Azure OpenAI Endpoint URL(AZURE_OPENAI_ENDPOINT) 및 API Key(AZURE_OPENAI_API_KEY)를 넣습니다.  
만약 하단 코드가 정상적으로 수행되지 않을 경우, 해당 파일을 저장 후 파일을 닫은 후 새로 열어서 진행합니다. (커널 다시 시작을 해도 됩니다. 상단의 `재시작` 버튼 클릭)

In [1]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

## 3. 작업에 적합한 OpenAI 모델 선택
2024년 12월 기준, 본 실습에서 다음의 모델을 사용합니다. 실습의 편의를 위해 모델을 배포할 때에는 아래 이름과 동일하게 배포하는 것을 추천 드립니다.  
- LLM 모델: `gpt-4o-mini`  
- 임베딩 모델: `text-embedding-3-large`  

`gpt-4o` 와 `gpt-4o-mini` 모델은 한글 토크나이저의 최적화로 기존 `gpt-4` 대비 더 빠르고 정확한 성능에 저렴한 비용을 제공합니다.  
모델에 자세한 정보는 다음을 참고하세요: [Azure OpenAI Service models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)  

In [2]:
deployment_name = os.getenv("DEPLOYMENT_NAME")
deployment_embedding_name = os.getenv("DEPLOYMENT_EMBEDDING_NAME")

## 4. Prompt Design  

대형 언어 모델(LLM)의 마법은 방대한 양의 텍스트를 훈련함으로써, 다음에 나올 텍스트(토큰) 예측 오류를 최소화 하도록 만들어져 있습니다.
예를 들어 다음과 같은 개념을 학습합니다:

* 어떻게 쓰는지
* 문법의 작동 방식
* 의역하는 방법
* 질문에 대답하는 방법
* 대화를 이끄는 방법
* 여러 언어로 작성하는 방법
* 코딩하는 방법
* 기타

### 대형 언어 모델(LLM)을 제어하는 방법  
LLM에 대한 모든 입력 중에서 가장 영향력 있는 것은 **텍스트 프롬프트** 입니다.

대규모 언어 모델은 몇 가지 방법으로 출력을 생성하라는 메시지를 표시할 수 있습니다: 

* **지침(Instruction)**: 모델에게 원하는 지침을 설명하세요.  

* **완료(Completion)**: 모델이 원하는 것의 시작을 끝낼수 있도록 유도하십시오.  

* **시연(Demonstration)**: 다음 중 하나를 사용하여 원하는 것을 모델에 보여주십시오.  
프롬프트 내에 몇 가지 예시를 주거나, 또는 수백 또는 수천 개의 예로 구성된 훈련 데이터 세트를 이용한 미세 조정을 할 수 있습니다.  


### 프롬프트를 만들기 위한 세 가지 기본 지침

* **Show and tell**: 명확하고 간결한 지침을 포함하고, 예시 또는 두 가지 조합을 통해 원하는 것을 명확히 기술합니다.  

* **Provide quality data**: 양질의 예시를 제시함으로 원하는 결과물을 만들어 낼 수 있도록 유도합니다. 할루시네이션을 최소화 하는 가장 확실한 방법은 정확한 데이터를 전달하는 것으로 시작합니다. (RAG 패턴 활용)  

* **Check your setting**: 온도(Temperature) 및 TOP_P 설정은 모델이 응답을 생성하는 데 결정적인 방법을 제어합니다. 정답이 하나만 있는 응답을 요청하는 경우 온도를 더 낮게(0) 설정하고 싶을 것입니다. 더 다양한 응답을 찾고 있다면 더 창의적인 응답을 원할 때에는 온도를 높게(1) 설정할 수 있습니다.

In [3]:
# Create your first prompt
system_message = """너는 긍정과 부정을 구분할 수 있는 에이전트야. 결과는 JSON 형식으로 공백 없이 반환해줘. 답변 예:{"1": "긍정", "2": "부정"}"""
user_message = """1. 모니터가 너무 뜨거워. 2. 모니터가 시장 반응이 너무 뜨거워."""

## 5. Submit!

In [4]:
# Simple API Call
response = client.chat.completions.create(
    model=deployment_name,
    max_tokens=60,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]
)

response.choices[0].message.content

'{"1":"부정","2":"긍정"}'

### 출력물 데이터 분석 
해당 API의 결과 이외에 어떤 정보들이 포함되어져 있는지 확인해 봅시다.  

In [5]:
print(json.dumps(response.model_dump(), indent=2))

{
  "id": "chatcmpl-AfnbKMqnn0pE6F1JfILjOb7Yq2nyW",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "{\"1\":\"\ubd80\uc815\",\"2\":\"\uae0d\uc815\"}",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null,
        "refusal": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
          "detected": false
        },
        "protected_material_text": {
          "filtered": false,
          "detected": false
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1734525478

### Azure OpenAI 사용 비용 확인
여기에는 gpt-4o 시리즈를 사용했을 때의 [비용](https://azure.microsoft.com/ko-kr/pricing/details/cognitive-services/openai-service/#pricing)을 출력합니다.  
비교 버전은 gpt-4o(0806) 과 gpt-4o-mini(0718) 입니다.  
배포된 모델의 버전에 따라 비용은 차이가 발생할 수 있습니다.  
(AOAI는 지속적으로 비용을 감소하는데 노력하고 있습니다.)

| model | version | prompt (1k) | completion (1k) |
|-----|-----|-----|-----|
| gpt-4o | 1120 | $0.0025 | $0.0100 |
| gpt-4o | 0806 | $0.0025 | $0.0100 |
| gpt-4o | 0513 | $0.0050 | $0.0150 |
| gpt-4o-mini | 0718 | $0.00015 | $0.0006|

gpt-35-turbo는 gpt-4o(0513) 대비 1/10 수준입니다.  
gpt-4o-mini는 gpt-4o(0513) 대비 1/30 수준입니다.  
아래는 gpt-4o를 활용할 때 비용을 산출하는 예시입니다.

In [6]:
# OpenAI API 사용 비용 계산 함수
def openai_cost(model_name):

    response = client.chat.completions.create(
        model=deployment_name,
        max_tokens=60,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ]
    )

    print(response.choices[0].message.content)

    dollar_to_won = 1400
    if model_name == "gpt-4o":
        model_prompt_cost = 0.0025
        model_complet_cost = 0.01
    elif model_name == "gpt-4o-mini":
        model_prompt_cost = 0.00015
        model_complet_cost = 0.0006
    else:
        print("선택된 모델이 없습니다.")

    print("프롬프트 토큰수:", response.usage.prompt_tokens)
    print("컴플리션 토큰수:", response.usage.completion_tokens)
    print("모든 토큰수:", response.usage.total_tokens)

    prompt_cost  = round(response.usage.prompt_tokens * model_prompt_cost / 1000 * dollar_to_won, 2)
    complet_cost = round(response.usage.completion_tokens * model_complet_cost / 1000 * dollar_to_won, 2)

    print(deployment_name, "사용에 따른 비용: ", prompt_cost, "원 + ", complet_cost, "원 = ", prompt_cost + complet_cost , "원" )

    return prompt_cost + complet_cost;


### gpt-4o-mini 모델을 호출합니다.
Azure OpenAI Stduio에서 `gpt-4o-mini` 모델이 배포되지 않았을 경우, 에러가 발생할 수 있습니다.  
에러가 발생할 경우, `gpt-4o-mini` 라는 이름으로 모델이 배포 되어 있는지 테스트 해 보세요.

In [7]:
deployment_name = "gpt-4o-mini"
gpt_4o_mini_pricing = openai_cost(deployment_name)

{"1":"부정","2":"긍정"}
프롬프트 토큰수: 91
컴플리션 토큰수: 12
모든 토큰수: 103
gpt-4o-mini 사용에 따른 비용:  0.02 원 +  0.01 원 =  0.03 원


### 이번에는 gpt-4o 모델을 호출합니다.
Azure OpenAI Stduio에서 `gpt-4o` 모델이 배포되지 않았을 경우, 에러가 발생할 수 있습니다.  
에러가 발생할 경우, `gpt-4o` 라는 이름으로 모델이 배포 되어 있는지 테스트 해 보세요.  
(버전에 따라 비용이 달라질 수 있으나, 여기서는 0806 버전 기준으로 비용을 표시합니다.)

In [8]:
deployment_name = "gpt-4o"
gpt_4o_pricing = openai_cost(deployment_name)

{"1":"부정","2":"긍정"}
프롬프트 토큰수: 91
컴플리션 토큰수: 12
모든 토큰수: 103
gpt-4o 사용에 따른 비용:  0.32 원 +  0.17 원 =  0.49 원


gpt-4o와 gpt-4o-mini 비용은 한글 토큰나이저의 최적화로 인하여 더욱 저렴하고 빠르게 동작합니다.

## 다양한 케이스에 대한 활용법
1. 요약
2. 분류
3. 제품 이름 생성
4. 임베딩

### 1. 요약
LLM은 다양한 케이스에서 사용이 가능합니다. 요약하기 위한 방법은 다음과 같습니다.
텍스트에 대한 토큰 길이를 확인할 수 있는 방법:
1. 토크나이저: https://platform.openai.com/tokenizer 

In [9]:
original_text = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"
system_message = "Summarize the text below as a bullet point list of the most important points using Korean."
user_message = "Text: ```" + original_text + "```"

In [10]:
# deployment_name = os.getenv("DEPLOYMENT_NAME")

#Setting a few additional, typical parameters during API Call
response = client.chat.completions.create(
  model=deployment_name,
  messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
  ],
  temperature=0.7,
  max_tokens=300
)

print(response.choices[0].message.content)

- 대규모 텍스트 코퍼스에서 사전 학습 후 특정 작업에 대한 미세 조정을 통해 NLP 작업과 벤치마크에서 큰 성과를 달성.  
- 기존 방법은 작업에 무관한 아키텍처를 사용하지만, 여전히 수천에서 수만 개의 예제가 포함된 작업별 미세 조정 데이터셋 요구.  
- 인간은 일반적으로 몇 가지 예제나 간단한 지시만으로 새로운 언어 작업 수행 가능.  
- 현재 NLP 시스템은 이러한 능력에서 여전히 부족함.  
- 언어 모델의 규모를 확장하면 작업에 무관한 소수 샘플 기반 성능(few-shot performance)이 크게 향상됨.  
- 때로는 이전 최첨단 미세 조정 접근법과 경쟁할 수준에 도달하기도 함.  


#### 참고하는 Text 문서가 없을 때
아래와 같이 참고하는 Text 문서가 없을 때에 어떻게 결과가 출력하는지 살펴 봅니다. RAG에서 다음과 같이 모델 자체 지식이 아닌 전달 받은 데이터 기반으로 출력하도록 구성할 수 있습니다.

In [11]:
original_text = ""
system_message = "Summarize the text below as a bullet point list of the most important points using Korean."
user_message = "Text: ```" + original_text + "```"

#Setting a few additional, typical parameters during API Call
response = client.chat.completions.create(
  model=deployment_name,
  messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
  ],
  temperature=0.7,
  max_tokens=300
)

print(response.choices[0].message.content)

요청하신 텍스트를 제공해 주시면 요약을 도와드리겠습니다!


### 2. 분류

제공된 범주에 맞게 적절한 항목 분류를 추론합니다. 다음 예에서는 프롬프트에서 분류 할 범주와 텍스트를 모두 제공합니다.

고객 문의 : 안녕하세요, 최근 노트북 키보드의 열쇠 중 하나가 최근에 파산되었으며 교체가 필요합니다.

분류 카테고리 :

In [12]:
system_message = """Classify the following inquiry into one of the following:
categories: [Pricing, Hardware Support, Software Support]
"""
user_message = """inquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement.
Classified category:
"""

In [13]:
response = client.chat.completions.create(
  model=deployment_name,
  messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
  ],
  temperature=0,
  max_tokens=60
)

print(response.choices[0].message.content)

Hardware Support


### 3. 제품 이름 생성
시드 단어를 기반으로 제품 이름을 만듭니다. 여기에는 이름을 생성할 제품에 대한 프롬프트 정보가 포함되어 있습니다. 또한 생성 패턴을 보여주는 비슷한 예를 제공합니다. 보다 창의적인 반응성을 높이기 위해 온도 값을 높게 설정했습니다.

제품 설명 : 발 크기에 맞는 신발 한 쌍.  
종자 단어 : 적응성, 적합, 옴니 피트.  

> temperature 값이 0에서 1 사이로 변경하여 반복 실행시 어떤 변화가 있는지 확인 합니다.

In [14]:
system_message = """Come up with five product names that fit the given product description and seed words."""
user_message = """Product description: A pair of shoes that can fit any foot size.
Seed words: adaptable, fit, omni-fit.
Product names:"""

In [15]:
response = client.chat.completions.create(
  model=deployment_name,
  messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
  ],
  temperature=0.8,
  # temperature=0,
  max_tokens=60
)

print(response.choices[0].message.content)

1. **AdaptiSole**  
2. **FlexiFit Pro**  
3. **OmniStride**  
4. **UniFit Glide**  
5. **MorphoStep**  


### 4. 임베딩
이 섹션에서는 임베딩을 검색하고 단어, 문장 및 문서 사이의 유사성을 찾는 방법을 보여줍니다.  
text-embedding-ada-002 API의 경우 기본 디멘션은 1,536 차원이며, text-embedding-3-large의 경우 2배인 3,072 차원입니다.  
(Update 2024.12: text-embedding-ada-002 에서 text-embedding-3-large 로 임베딩 API를 변경하였습니다.)

In [16]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [17]:
text = 'the quick brown fox jumped over the lazy dog'

In [18]:
vector = client.embeddings.create(input = [text], model=deployment_embedding_name).data[0].embedding
print(vector)
len(vector)

[-0.005379660055041313, -0.006699045188724995, 0.0018830852350220084, -0.001022727694362402, 0.007556849624961615, -0.028103310614824295, 0.008390145376324654, 0.06751330196857452, -0.013602329418063164, 0.02217220515012741, 0.00883130170404911, 0.032122738659381866, 0.0074302214197814465, -0.045096009969711304, -0.018626613542437553, 0.03297237306833267, -0.04244907200336456, 0.01620842143893242, 0.004954842384904623, 0.013520633801817894, 0.010979898273944855, -0.028609823435544968, -0.04097854718565941, 0.019803030416369438, -0.01885536126792431, 0.018790004774928093, -0.005191759672015905, 0.009983210824429989, 0.009550224058330059, 0.007675308268517256, 0.00872509740293026, 0.03217175602912903, -0.01211546827107668, 0.010530571453273296, 0.030162042006850243, 0.0007852996350266039, 0.024672092869877815, 0.0012560709146782756, -0.004452413879334927, 0.00523260748013854, 0.031861312687397, -0.013153002597391605, -0.026273326948285103, -0.018005724996328354, -0.012352385558187962, 0.

3072

In [19]:
sentences1 = ['The new movie is awesome',  
              'The new movie is awesome',  
              'The new movie is awesome']  
  
sentences2 = ['The dog plays in the garden',  
              'This recent movie is so good',  
              'The new movie is awesome']  

embeddings1 = [client.embeddings.create(input = s, model=deployment_embedding_name).data[0].embedding for s in sentences1]  
embeddings2 = [client.embeddings.create(input = s, model=deployment_embedding_name).data[0].embedding for s in sentences2]  
  
for i in range(len(sentences1)):  
    print("{}\t{}\tScore: {:.4f}".format(sentences1[i], sentences2[i], cosine_similarity(embeddings1[i], embeddings2[i])))

The new movie is awesome	The dog plays in the garden	Score: 0.1551
The new movie is awesome	This recent movie is so good	Score: 0.6300
The new movie is awesome	The new movie is awesome	Score: 1.0000


다음은 한글 문장을 임베딩 API를 이용하여 벡터 값으로 바꾼 후 비교한 결과입니다.  
유사도 점수가 어떤 차이가 날 수 있는지를 확인 합니다.

In [20]:
sentences1 = [
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
            ]
  
sentences2 = [
              '바나나 우유는 목욕 후에 마시면 더 맛있습니다.',  
              '바나나 우유는 후에 목욕 마시면 더 맛있습니다.',  
              '목욕 후에 바나나 우유를 마시면 더 맛있습니다.',
              '맛있습니다 더 마시면 후에 목욕 우유는 바나나',  
              '딸기 우유는 목욕 전에 마시면 더 맛있습니다.',  
              '우유는 시리얼이랑 먹으면 더 고소한 맛이 납니다.',
              '어제는 비랑 눈이 와서 날씨가 더 추웠습니다.',
              '다가오는 2024 OpenAI DevDay는 직관할 수 있을까?',
              'ABCDE FGHI JKL MNO PQR STU VWXYZ ABCDEFG',
            ]

embeddings1 = [client.embeddings.create(input = s, model=deployment_embedding_name).data[0].embedding for s in sentences1]  
embeddings2 = [client.embeddings.create(input = s, model=deployment_embedding_name).data[0].embedding for s in sentences2]  
  
for i in range(len(sentences1)):  
    print("{}\t{}\tScore: {:.4f}".format(sentences1[i], sentences2[i], cosine_similarity(embeddings1[i], embeddings2[i])))

바나나 우유는 목욕 후에 마시면 더 맛있습니다.	바나나 우유는 목욕 후에 마시면 더 맛있습니다.	Score: 1.0000
바나나 우유는 목욕 후에 마시면 더 맛있습니다.	바나나 우유는 후에 목욕 마시면 더 맛있습니다.	Score: 0.9692
바나나 우유는 목욕 후에 마시면 더 맛있습니다.	목욕 후에 바나나 우유를 마시면 더 맛있습니다.	Score: 0.9016
바나나 우유는 목욕 후에 마시면 더 맛있습니다.	맛있습니다 더 마시면 후에 목욕 우유는 바나나	Score: 0.7544
바나나 우유는 목욕 후에 마시면 더 맛있습니다.	딸기 우유는 목욕 전에 마시면 더 맛있습니다.	Score: 0.6921
바나나 우유는 목욕 후에 마시면 더 맛있습니다.	우유는 시리얼이랑 먹으면 더 고소한 맛이 납니다.	Score: 0.4663
바나나 우유는 목욕 후에 마시면 더 맛있습니다.	어제는 비랑 눈이 와서 날씨가 더 추웠습니다.	Score: 0.1795
바나나 우유는 목욕 후에 마시면 더 맛있습니다.	다가오는 2024 OpenAI DevDay는 직관할 수 있을까?	Score: 0.0575
바나나 우유는 목욕 후에 마시면 더 맛있습니다.	ABCDE FGHI JKL MNO PQR STU VWXYZ ABCDEFG	Score: 0.0128


text-embedding-ada-002의 경우, 0.6x부터 1까지의 코사인 유사도 값이 나옵니다.  
text-embedding-3-large의 경우, 0부터 1까지의 코사인 유사도 값이 나옵니다.

# 궁금증 있을 때 연락 주세요.  
Prompt Engineering과 관련 궁금증이 있을 경우 아래 이메일로 요청 주세요.  
MS Korea 김현수 이메일: [<hyounsookim@microsoft.com>](hyounsookim@microsoft.com)